Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Xinyu Meng"
COLLABORATORS = ""

---

<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself. We will NOT grade this submissing w/o all the information<br>
<b>Name</b>:Xinyu<br>
<b>NetID</b>:xm73<br>
<b>Recitation (01,02)</b>:02<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:X.M      


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

## CS 439 - Introduction to Data Science
### Fall 2022

# Lab 3: Exploratary Data Analysis

### Due Date: Sunday October 23, 2022 by 11:59 PM ###
#### 60 points ####

### Instructions
This lab is presented as a notebook. Please execute the cells that are already completed and your task is to fill in the code
between ### START SOLUTION ### and ### END_OF SOLUTION ###. 

#### Important: Please do not add any new cells or change the order of cells. If you have questions, please contact the courseS staff.

In this lab, you will be working with a dataset from NYPD containing data on calls to the New York Police Department. Information about the datasets can be found at https://opendata.cityofnewyork.us/

## Setup

Note that after activating matplotlib to display figures inline via the IPython magic `%matplotlib inline`, we configure a custom default figure size. Virtually every default aspect of matplotlib [can be customized](https://matplotlib.org/users/customizing.html).

In [ ]:
import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

# Part 1 : Getting Data (15 pts)

We will work with the NYPD Historic complaint data set. Our first task is to estimate the size of this download by looking at the number of rows, columns and using an estimated size for a column (use a reasonable value). The site metadata is available from the page
https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
### START SOLUTION
#estimated size of the download (based on metadata information from the site) = 1824.39 mb
# please explain how you reached the answer
#We assume the numbers are int32 each takes 4 bytes and each text takes 10 bytes, datetime takes 8 bytes, the location are variables
#we assume it to be 8 byte each.
#There are 7.83m rows, each row has 10 int, 21 text, 3 datetime and 1 location so 7.83*(4*10+10*21+8*3+8*1) = 2160.25mb
### END_OF SOLUTION

### 1.1 Download the data
This file is large (use the estimate you did above). If it takes too long to download, you may want to interrupt and download the file using a browser and URL https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i

In [ ]:
import utils
data_dir = 'data'
data_url = 'https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i'

file_name = 'NYPD_Complaint_Data_Historic.csv'

# To retrieve the dataset, we will use the `utils.fetch_and_cache` utility from utils library. 
dest_path = utils.fetch_and_cache(data_url, file_name, data_dir)
print(f'Located at {dest_path}')

### 1.2 Inspect the size of the file
It is helpful to get an idea of the size of the file. This can be done using functions in the utils library.

In [ ]:
# look at the size of the file w/o opening it using OS (https://docs.python.org/3/library/os.html). You can perform
# variety of operating system related functions from this package.
import os
### START SOLUTION
size = os.path.getsize('data/NYPD_Complaint_Data_Historic.csv')
print(size)
### END_OF SOLUTION

### 1.3 Split the large file
This data file NYPD_Complaint_Data_Historic.csv is too big to load into a single DataFrame. Let us split the large file into smaller files.  Let us find out the number of lines in the NYPD_Complaint_Data_Historic.csv file using utils.

In [ ]:
# Using utils library, find the number of lines in the file

### START SOLUTION
number_of_lines = utils.line_count('data/NYPD_Complaint_Data_Historic.csv')
print(number_of_lines)
### END_OF SOLUTION

In [ ]:
# Split the file into 10 smaller files. Estimate the number of lines in each file using the count above
# files should be created in the data folder and named NYPD_Complaint_Data_Historic_1.csv, 
# NYPD_Complaint_Data_Historic_2.csv, ... NYPD_Complaint_Data_Historic_10.csv etc
# It is possible that few lines from the original file may not be saved due to rounding errors.

### START SOLUTION

source_path = "data/NYPD_Complaint_Data_Historic.csv"
for i,chunk in enumerate(pd.read_csv(source_path, chunksize=177315), 1):
    chunk.to_csv('data/NYPD_Complaint_Data_Historic_{}.csv'.format(i), index=False)
    print(utils.line_count('data/NYPD_Complaint_Data_Historic_{}.csv'.format(i)))

### END_OF SOLUTION

### 1.4 Check the files in the data folder
Now, we'll use a method of the `Pathlib.Path` class called `glob` to list all files in the `data` directory. You will find useful information in pathlib [docs](https://docs.python.org/3/library/pathlib.html).

Below, we use pathlib's `glob` method to store the list of all files' names from the `data_dir` directory in the variable `file_names`. These names should be strings that contain only the file name (e.g. `dummy.txt` not `data/dummy.txt`). The asterisk (*) character is used with the `glob` method to match any string.

In [ ]:
from pathlib import Path
data_dir_path = Path('data') # creates a Path object that points to the data directory
file_names = [x.name for x in data_dir_path.glob('*') if x.is_file()]
file_names

### 1.5 Pre-processing of data
It is good to pre-process the data to see if the file can be opened in a Jupyter notebook. We need to avoid large files that can crash notebooks. Typically, files of size around 200 MB is ok to open into a DataFrame. In the following activities we will inspect the file w/o opening it as a DataFrame. Using utils.head 

In [ ]:
# Use the util.head() to read up to 5 lines from the original file (w/o opening it)
from utils import head
head('data/NYPD_Complaint_Data_Historic.csv')
allDF = pd.read_csv("data/NYPD_Complaint_Data_Historic.csv")

### 1.6 Inspecting and describing data columns
There should be 35 columns in each record. Using header information and data types, describe the type of data in each
column. If you are unable to determine, just state so.
##### START SOLUTION
* CMPLNT_NUM : int64
* CMPLNT_FR_DT: object
* CMPLNT_FR_TM: object 
* CMPLNT_TO_DT: object
* CMPLNT_TO_TM: object
* ADDR_PCT_CD: float64
* RPT_DT: object
* KY_CD: int64
* OFNS_DESC: object
* PD_CD: float64
* PD_DESC: object
* CRM_ATPT_CPTD_CD: object
* LAW_CAT_CD: object
* BORO_NM: object
* LOC_OF_OCCUR_DESC: object
* PREM_TYP_DESC: object
* JURIS_DESC: object
* JURISDICTION_CODE: float64
* PARKS_NM: object
* HADEVELOPT: object
* HOUSING_PSA: object
* X_COORD_CD: float64
* Y_COORD_CD: float64
* SUSP_AGE_GROUP: object
* SUSP_RACE: object
* SUSP_SEX: object
* TRANSIT_DISTRICT: float64
* Latitude: float64
* Longitude: float64
* Lat_Lon: object
* PATROL_BORO: object
* STATION_NAME: object
* VIC_AGE_GROUP: object
* VIC_RACE: object
* VIC_SEX: object
##### END_OF SOLUTION

# Part 2 - Exploratory Data Analysis  (20 points)
Exploratory data analysis (EDA) is the process of examining a subest of a large data set to see what we can know about the data. First we will explore one file NYPD_Complaint_Data_Historic_1.csv to see what we can find out.

### 2.1 Time to load data into a DataFrame
Load the first CSV file, NYPD_Complaint_Data_Historic_1.csv into a `pandas.DataFrame` object. Also do a time analysis to see how long it took to load the data into a DataFrame. Time should be printed in seconds. The time libraries https://docs.python.org/3/library/time.html can help.

In [ ]:
### START SOLUTION
import pandas as pd
from time import perf_counter
start = perf_counter()
calls = pd.read_csv("data/NYPD_Complaint_Data_Historic_USE_THIS.csv")
print(f"{perf_counter()-start:0.2f} s elapsed")
### END_OF SOLUTION

### 2.2 Description of Fields
Let's also check some basic information about these files using the `DataFrame.describe` and `DataFrame.info` methods. Describe columns that can be removed based on the information.

In [ ]:
### START SOLUTION
calls.info()
calls.describe()

# What columns can be removed from the DataFrame? A reasonable rule of thumb is that if a column is missing more than 
# 50% of the data then it should be removed
# PARKS_NM, HADEVELOPT, HOUSING_PSA, TRANSIT_DISTRICT, STATION_NAME can be removed from the dataframe.
### END_OF SOLUTION

### 2.2 Finding Uniques

Notice that the functions above reveal type information for the columns, as well as some basic statistics about the numerical columns found in the DataFrame. However, we still need more information about what each column represents. Let's explore the data further.

find the number of unique values in each DataFrame column and answer the questions below.

In [ ]:
### START SOLUTION
calls.nunique()

# Questions
# 1. How many distinct locations where the complaints have come from?
# 49146
# 2. How many age groups are represented in the data set?
# 17
# 3. How many boroughs are included in the data set?
# 5
# 4. How many offense types are listed in this data set?
# 62

### END_OF SOLUTION

### 2.3 Offense by Boro
Using GroupBy operation, create a DataFrame that groups offenses by Boro. call the DataFrame calls_by_Boro_and_offense

In [ ]:
# YOUR CODE HERE
calls_by_Boro_and_offense = calls.groupby(['BORO_NM'])
calls_by_Boro_and_offense.groups

### 2.4 Offenses in Bronx

In the cell below, find a list of strings corresponding to the possible values for `OFNS_DESC` when `BORO` is "BRONX". Create an expression that automatically extracts the names of the offenses.

In [ ]:
# YOUR CODE HERE
calls_by_Boro_and_offense.get_group('BRONX').OFNS_DESC.unique()

##### Question: How many offenses were committed in Bronx during the analysis period? Write the answer below.

There are 38506 offenses cases commited in Bronx

### 2.5 Most Common Crimes in NYC

What are the five crime types of OFNS_DESC that have the most crime events in Bronx? You may need to use `value_counts` to find the answer. Save your results as a list of strings.

**Hint:** *The `keys` method of the Series class might be useful.*

In [ ]:
# YOUR CODE HERE
Most_Common_Crimes = calls_by_Boro_and_offense.get_group('BRONX')['OFNS_DESC'].value_counts().head().keys()
Most_Common_Crimes

### 2.6 Probability of a Crime in Bronx
What is the probability that a the crime "Arson" can happen in Bronx?

In [ ]:
### START SOLUTION
Bronx = calls_by_Boro_and_offense.get_group('BRONX')
probability = len(Bronx[Bronx['OFNS_DESC'] == 'Arson'])/len(Bronx['OFNS_DESC'])
probability
### END_OF SOLUTION

---
# Part 3: Visualizing the Data (20 pts)

## Pandas vs. Seaborn Plotting

Pandas offers basic functionality for plotting. For example, the `DataFrame` and `Series` classes both have a `plot` method. However, the basic plots generated by pandas are not particularly pretty. While it's possible to manually use matplotlib commands to make pandas plots look better, we'll instead use a high level plotting library called Seaborn that will take care of most of this for us.

As you learn to do data visualization, you may find the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.plot.html) and [Seaborn documentation](https://seaborn.pydata.org/api.html) helpful!

We will continue use EDA for examining a subest of a large data set to see what we can know about the data. Continue to explore one file NYPD_Complaint_Data_Historic_1.csv to see what we can find out.

### 3.1 Plotting a Series
Using the built-in plotting functionality of pandas, such as `plot` method of the `Series` class to generate a `barh` plot type,  display the value counts for `OFNS_DESC` visually as a barh chart.

In [ ]:
### START SOLUTION
calls_by_Boro_and_offense.get_group('BRONX')['OFNS_DESC'].value_counts().plot.barh();

### END_OF SOLUTION

### 3.2 Getting a Better Plot
The plot above can be messy as it plots all offenses. Plot only the offenses that has more than 10000 calls

In [ ]:
### START SOLUTION
temp = calls_by_Boro_and_offense.get_group('BRONX')['OFNS_DESC'].value_counts().to_frame()
ax = temp.loc[temp['OFNS_DESC'] > 10000].squeeze().plot.barh()
ax.set(xlabel = "Number of Calls", ylabel = "Crime Category", title = "Number of Calls By Crime Type");
### END_OF SOLUTION

By contrast, the Seaborn library provides a specific function `countplot` built for plotting counts. It operates directly on the DataFrame itself i.e. there's no need to call `value_counts()` at all. This higher level approach makes it easier to work with. Use the y-label ("Crime Category"), x-label("Number of Calls") and title_of_plot("Number of Calls By Crime Type"

In [ ]:
### START SOLUTION
ax = sns.countplot(y = 'OFNS_DESC', data = calls['OFNS_DESC'].to_frame())
ax.set(xlabel = "Number of Calls", ylabel = "Crime Category", title = "Number of Calls By Crime Type")
plt.show();
### END_OF SOLUTION

You may note that the ordering might be different for the seaborn plot (as compared to pandas plot). If we want the same ordering that we had in the pandas plot, we can use the order parameter of the `countplot` method. It takes a list of strings corresponding to the axis to be ordered. By passing the index of the `value_counts`, you can get the order you want.

In [ ]:
### START SOLUTION
ind = calls_by_Boro_and_offense.get_group('BRONX')['OFNS_DESC'].value_counts(ascending = True).index
ax = sns.countplot(y = 'OFNS_DESC', 
                   data = calls_by_Boro_and_offense.get_group('BRONX')['OFNS_DESC'].to_frame(), 
                   order = list(ind))
ax.set(xlabel = "Number of Calls", ylabel = "Crime Category", title = "Number of Calls By Crime Type")
plt.show();
### END_OF SOLUTION

Now we have a pretty bar plot with the bars ordered by size. Though `seaborn` appears to provide a superior plot from a aesthetic point of view, the `pandas` plotting library is also good to understand. You'll get practice using both libraries in the following questions.

## An Additional Note on Plotting in Jupyter Notebooks

You may have noticed that many of our code cells involving plotting end with a semicolon (;). This prevents any extra output from the last line of the cell that we may not want to see. Try adding this to your own code in the following questions!

### 3.3 making more plots

Now it is your turn to make some more plots using `pandas` and `seaborn`. Let's start by looking at the distribution of calls over days of the week.

The CMPLNT_FR_DT field contains the date of the event. We would like to add a new column to the DataFrame that includes Day of the week (DAY_OF_WEEK) that indicates the day of the week. This can help us analyze the crimes on a specific day of the week. For example, we can answer questions such as "what day of the week that a LARSON is likely to happen in NYC?"


Add a new column `DAY_OF_WEEK` into the `calls` dataframe that has the day string (eg. 'Sunday') for the corresponding value in CMPLNT_FR_DT. For example, if the first 3 values of `CMPLNT_FR_DT` are `['01/27/2006, '01/28/2006, '01/29/2006]`, then the first 3 values of the `DAY_OF_WEEK` column should be `["Friday", "Saturday", "Sunday"]`.

**Hint:** *Try using the [Series.map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function on `calls["OFNS_DESC"]`.  Can you assign this to the new column `calls["DAY_OF_WEEK"]`?*

In [ ]:
# YOUR CODE HERE
from datetime import datetime
from dateutil import parser
import calendar

temp = calls.copy()
temp['DAY_OF_WEEK'] = temp.apply(lambda x: calendar.day_name[parser.parse(x['CMPLNT_FR_DT']).weekday()] if type(x['CMPLNT_FR_DT']) == str else x['CMPLNT_FR_DT'], axis = 1)
calls = temp
calls 

### 3.4 Seaborn plots

Create a `seaborn` plot that shows the number of calls for each day of the week. You may want to use of the `rotation` argument in `ax.set_xticklabels`, which rotates the labels by 90 degrees.

In [ ]:
### START SOLUTION
days_sequence = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 
                 'Friday', 'Saturday']
ax = sns.countplot(x = 'DAY_OF_WEEK', 
                   data = calls['DAY_OF_WEEK'].to_frame(), order = days_sequence)
ax.set(ylabel = "Number of Calls", 
       title = "Number of Calls For Each Day of the Week")
plt.xticks(rotation=90)
plt.show();
### END_OF SOLUTION

Now, let's make the same plot using `pandas`. Construct a vertical bar plot with the count of the number of calls (entries in the table) for each day of the week **ordered by the day of the week** (eg. `Sunday`, `Monday`, ...). Do not use `sns` for this plot. Be sure that your axes are labeled and that your plot is titled.

**Hint:** *Given a series `s`, and an array `coolIndex` that has the same entries as in `s.index`, `s[coolIndex]` will return a copy of the series in the same order as `coolIndex`.*

In [ ]:
# YOUR CODE HERE
ax = calls['DAY_OF_WEEK'].value_counts()[days_sequence].plot.bar()
ax.set(ylabel = "Number of Calls", 
       title = "Number of Calls For Each Day of the Week", xlabel = "Day")
plt.show()

### 3.5 What Day of the Week is more calls?

Is it true that weekdays generally have slightly more calls than Saturday or Sunday? What can you say about the difference?

##### START SOLUTION
Yes, it's true that weekdays generally have more calls than Saturday or Sunday but the difference would be very small. Since in weekdays, people will have more commuting activities in the neighborhood, and in the process, people will experience more crime and result in generally more calls to the police.
##### END_OF SOLUTION

We can break down into some particular types of events to see their distribution. For example, let's make a bar plot for the OFNS_DESC "HARRASSMENT 2". Which day is the peak for "HARRASSMENT 2"?

This time, use `seaborn` to create a vertical bar plot of the number of total noise violations reported on each day of the week, again ordered by the days of the week starting with Sunday. Do not use `pandas` to plot.

**Hint:** *If you're stuck, use the code for the seaborn plot in above question as a starting point.*

In [ ]:
# YOUR CODE HERE
ax = sns.countplot(x = 'DAY_OF_WEEK', 
                   data = calls[calls['OFNS_DESC'] == 'HARRASSMENT 2']['DAY_OF_WEEK'].to_frame(), order = days_sequence)
ax.set(ylabel = "Number of Calls", 
       title = "Number of Harrassment-2 Calls For Each Day of the Week")
plt.xticks(rotation=90)
plt.show();

### 3.6 Distribution

Do you see anything interesting about the distribution of HARRASSMENT 2 calls over a week? Type a short answer below.
##### START SOLUTION
The number of cases of harrassment2 in Saturday is the least over a week, the number of calls keeps increasing from Sunday to Wednesday, and the number reaches its max value at Wednesday.
##### END_OF SOLUTION

### 3.7 More Plots

Let's look at a similar distribution but for a crime we have much more calls data about. In the cell below, create the same plot as you did in previous questions, but now looking at instances of the OFNS_DESC "BURGLARY" (instead of "HARRASSMENT 2"). Use either `pandas` or `seaborn` plotting as you desire.

In [ ]:
# YOUR CODE HERE
ax = sns.countplot(x = 'DAY_OF_WEEK', 
                   data = calls[calls['OFNS_DESC'] == 'BURGLARY']['DAY_OF_WEEK'].to_frame(), order = days_sequence)
ax.set(ylabel = "Number of Calls", 
       title = "Number of BURGLARY Calls For Each Day of the Week")
plt.xticks(rotation=90)
plt.show();

### 3.8 time of events

Now let's look at the CMPLNT_TO_TM column which indicates the time for events. Since it contains hour and minute information, let's extract the hour info and create a new column named `Hour` in the `calls` dataframe. You should save the hour as an `int`. Then plot the frequency of each hour in the table (i.e., `value_counts()`) sorted by the hour of the day (i.e., `sort_index()`).

You will want to look into how to use:

* [Series.str.slice](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.slice.html#pandas.Series.str.slice) to select the substring.
* [Series.astype](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) to change the type.

**Hint:** *The `str` helper member of a series can be used to grab substrings.  For example, `calls["CMPLNT_TO_TM"].str.slice(3,5)` returns the minute of each hour of the `CMPLNT_TO_TM`.*

In [ ]:
# YOUR CODE HERE
#astype not working?
calls['Hour'] = calls["CMPLNT_TO_TM"].str.slice(0,2)
calls['Hour'] = calls['Hour'].replace(np.NaN, 0).astype(float).astype(int)
hour_sequence = [x for x in range(24)]
ax = calls['Hour'].value_counts()[hour_sequence].plot.bar()
ax.set(ylabel = "Number of Calls", 
       title = "Number of Calls per Hour", 
       xlabel = "Hour")
plt.show();

Create a pandas bar plot showing the number of BURGLARY crimes committed at each hour of the day. Use the labels
* ax.set_xlabel("Hour of the Day")
* ax.set_ylabel("Number of Calls")
* ax.set_title("Number of Calls Reporting Fraud For Each Day of the Week");

In [ ]:
### START SOLUTION
ax = calls[calls['OFNS_DESC'] == 'BURGLARY']['Hour'].value_counts()[hour_sequence].plot.bar()
ax.set_xlabel("Hour of the Day")
ax.set_ylabel("Number of Calls")
ax.set_title("Number of Calls Reporting Burglary For Each Day of the Week");
### END_OF SOLUTION

### 3.9 More plots

In the cell below, create a seaborn plot of the same data. Again, make sure you provide axes labels and a title for your plot.

In [ ]:
# YOUR CODE HERE
ax = sns.countplot(x = 'Hour', 
                   data = calls[calls['OFNS_DESC'] == 'BURGLARY']['Hour'].to_frame())
ax.set_xlabel("Hour of the Day")
ax.set_ylabel("Number of Calls")
ax.set_title("Number of Calls Reporting Burglary For Each Day of the Week");

### 3.10 Spike in burglary?

According to your plots, is there a spike in calls reporting BURGLARY at any particular time? If so, Do you trust that this spike is legitimate, or could there be an issue with our data? Explain your reasoning in 1-2 sentences below.

#### START SOLUTION
There is a spike in calls reporting Burglary at around 8 am. This spike is legitimate because this situation is probably due to that from general knowledge, 8am is when many people wake up, and burglaries that occur during the long period of the night are more likely to be discovered by people when they wake up, which leads to a higher number of calls in the morning.

#### END_OF SOLUTION

In the cell below, we generate a boxplot which examines the hour of day of each crime broken down by the `OFNS_DESC` value.  To construct this plot we used the [DataFrame.boxplot](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.boxplot.html) documentation.

In [ ]:
calls.boxplot(column="Hour", by='OFNS_DESC', rot=90);

While the pandas boxplot is informative, we can use seaborn to create a more visually-appealing plot. Using seaborn, regenerate a better box plot. See either the textbook (https://www.textbook.ds100.org/ch/06/viz_quantitative.html) or the [seaborn boxplot documentation](https://seaborn.pydata.org/generated/seaborn.boxplot.html).

Looking at your plot, which crime type appears to have the largest interquartile range? Put your results into `answer` as a string.

In [ ]:
# YOUR CODE HERE
ax = sns.boxplot(data = calls, x = 'OFNS_DESC', y = 'Hour')
ax.set(title = 'Distributions of Calls Over Hours in a Day, Grouped by Crime')
plt.xticks(rotation=90)
plt.show()
answer = 'OFFENSES RELATED TO CHILDREN'

### 3.11a Explore Suspect and Victim Age Relationships
In this activity we explore relations between SUSP_AGE_GROUP and VIC_AGE_GROUP
1. create a matrix of where rows are SUSP_AGE_GROUPS and Columns are VIC_AGE_GROUPs
2. Fill in the matrix with all probabilities (eg. "the probability that <18 susp_group target <18 victim_group). 

In [ ]:
### START SOLUTION
pd.crosstab(calls['SUSP_AGE_GROUP'], calls['VIC_AGE_GROUP'], normalize='index')
#matrix = pd.DataFrame(index = calls['SUSP_AGE_GROUP'].value_counts().index, 
#                      columns = calls['VIC_AGE_GROUP'].value_counts().index)
#for index, row in matrix.iterrows():
#    for i in row.index:
#        matrix[i][index] = len(calls[(calls['VIC_AGE_GROUP'] == i) & (calls['SUSP_AGE_GROUP'] == index)]) / len(calls)
#matrix
### END_OF SOLUTION

### 3.11b Explore Suspect and Victim Sex Relationships
In this activity we explore relations between SUSP_SEX and VIC_SEX 
1. create a matrix of where rows are SUSP_SEX and Columns are VIC_SEX
2. Fill in the matrix with all probabilities (eg. "the probability that Males target other Males?")

In [ ]:
### START SOLUTION
pd.crosstab(calls['SUSP_SEX'], columns = calls['VIC_SEX'], normalize='index')

#matrix = pd.DataFrame(index = calls['SUSP_SEX'].value_counts().index, 
#                      columns = calls['VIC_SEX'].value_counts().index)
#for index, row in matrix.iterrows():
#    for i in row.index:
#        matrix[i][index] = len(calls[(calls['VIC_SEX'] == i) & (calls['SUSP_SEX'] == index)]) / len(calls)
#matrix
### END_OF SOLUTION

### 3.11c Observations 
Based on what was calculated in 3.11a and 3.11b, state at least important observations.
##### START SOLUTION
Seems like there are people with negative ages. Women targeting women have the highest probability. Also, female has the highest probability to be targeted for a crime.
##### END_OF SOLUTION

### 3.12 - Visualization of crimes on a Map of NYC
Finally we attempt to visualize the crimes committed in NYC on a Map. For this activity, use all data, not just the First set,  NYPD_Complaint_Data_Historic_1. 
First we need to install some mapping software. RSun the cell below to install folium package for mapping software.

In [ ]:
!/koko/system/anaconda/envs/python39/bin/pip install --user --upgrade folium

In [ ]:
### Plot the maps for BURGLARY in NYC
### if it takes too much time or map does not show up, try plotting a subset of the data set of for a specific crime

### START SOLUTION
import folium

temp = calls.loc[(calls['OFNS_DESC'] == 'BURGLARY') & (calls['SUSP_SEX'] == 'U') & (calls['DAY_OF_WEEK'] == 'Sunday')]
m = folium.Map(zoom_start=10)
locs = temp[['Latitude', 'Longitude']].astype('float').dropna()
locs.apply(lambda x: folium.Marker([x['Latitude'], x['Longitude']]).add_to(m), axis = 1)
m
### END_OF SOLUTION

# Part 4: Kernel Density Estimators (KDE) (5 pts)
When compared with the commonly used histogram, the kernel density estimator shows several advantages.
1. It is a smooth curve and thus it better exhibits the details of the probability density function.
2. It uses all sample points' locations, so, therefore, it better reveal the information contained in the sample.

In this activity, we will use KDE on CMPLNT_FR_DT data column from NYPD data set. Work with the column labled CMPLNT_FR_DT. The goal is to plot this one dimensional time data using KDE to see the distribution of time in a density map. You may have to convert time objects seconds.

### 4.1 - Using Seaborn kdeplot, plot the KDE graph

In [ ]:
### START SOLUTION
ax = sns.kdeplot(x = 'CMPLNT_FR_DT', 
                   data = pd.to_datetime(calls['CMPLNT_FR_DT'], errors = 'coerce').to_frame())
ax.set(xlabel = "CMPLNT_FR_DT")
plt.show()
### END_OF SOLUTION

### 4.2 - What can you deduce from the KDE plot? Write your responses here
##### START RESPONSE
The alarm calls in the data are all mainly concentrated around 2019-2020, while the data from 2010-2018 are more sparse, and the alarm calls data before 2010 are not recorded.

##### END_OF RESPONSE

### 4.3 (OPTIONAL - bonus - 5 pts) - Implement your own kernel function for KDE
Implement 3 Kernel functions from scratch from the following list.<br>
<img src='KDE.jpg' width="400" height="400"><br>
plot the KDE using your own code. Do not use kdeplot. Compare with Seaborn KDE plot. Do research to find out what kernel function that kdeplot is using and discuss advantages and disadvantages of each function compared to .

In [ ]:
### START SOLUTION
import math
def Biweight(t):
    if abs(t) >= 1:
        return 0
    else:
        return ((15/16) * (1-t**2))**2

def Triangular(t):
    if abs(t) < 1:
        return 1 - abs(t)
    else:
        return 0

def Gaussian(t):
    return (1/math.sqrt(2*math.pi)) * (math.e)**(-(t**2)/2)
### END_OF SOLUTION

<b> 4.3.1 </b> Plot the KDE using your own code. Do not use kdeplot. 

In [ ]:
### STAT SOLUTION
t = pd.to_datetime(calls['CMPLNT_FR_DT'], errors = 'coerce').dropna()
t = t.apply(lambda x: Gaussian(x.timestamp()))
sns.lineplot(data = t.to_frame())

### END_OF SOLUTION

Do your research to find what kernel function that kdeplot is using. Compare kdeplot density graph with your own.
### write your comments below
##### START RESPONSE


##### ENDOF RESPONSE

 ## Congratulations !!!

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> Output:</b> Please **remove all output** from your notebook prior to submission<br>
<b> File Name:</b> Please name the file as your_section_your_netID_Lab3.ipynb (eg. 01_adg133_Lab3.ipynb<br>
<b> Submit To: </b>CodeBench <br>
<b>Warning:</b> Failure to follow directions may result in loss of points.<br>
</div>

pd.crosstab(df['Product Type'], df['Lead'], normalize='all')@2022 A.D. Gunawardena. Many people contributed to this lab including TA Liqin Long (now at Google). Credits go to Josh Hug, and Berkeley Data Science Group for their contributions to the original version. Please DO NOT share this lab and/or post them on public sites such as github and others.